### Simple testing of the mflike likelihood
In this notebook we are going to show how to call the multi-frequency ACT likelkihood mflike and the lensing likelihood within SOLikeT and how to compare and combine the likelihoods.

Note that this interactive notebook repeats some of the steps in the tests folder of SOLikeT.



In [62]:
# The first thing we have to do is import solikeT and cobaya, 
# and tell cobaya where to look for the mflike installed likelihood
import soliket
import os
from cobaya.model import get_model
from soliket.tests.test_mflike import cosmo_params, nuisance_params
from packaging.version import Version
import camb
packages_path = os.environ.get("COBAYA_PACKAGES_PATH") or os.path.join(
    tempfile.gettempdir(), "LAT_packages"
)

Next up we define the first likelihood information so that it can be passed to cobaya easily. Note that the version of mflike supported here is v0.6 which is included in SOLikeT as opposed to v0.7.1 which is the current stand-alone mflike version. The two will be incorporated periodically.

In [63]:
info = {
    "packages_path": packages_path,
    "likelihood":  {"soliket.mflike.MFLike": 
                    {'data_folder': 'MFLike/v0.6/',
                     "input_file": 'data_sacc_00044.fits',
                     "cov_Bbl_file": "data_sacc_w_covar_and_Bbl.fits"}}}
# We will include the mflike likelihood
info['defaults']={
    "polarizations": {'TT', 'TE', 'ET', 'EE'},
    "scales": {"TT": '[2, 6002]',"TE": '[2, 6002]',"ET": '[2, 6002]',"EE": '[2, 6002]'},
    "symmetrize": 'False'}

info["theory"]= {
    "camb": {"extra_args": {"lens_potential_accuracy": 1}}, 
    "soliket.TheoryForge_MFLike": {"stop_at_error": True},
    "soliket.Foreground": {"stop_at_error": True},
    "soliket.BandPass": {"stop_at_error": True}
}
info["params"]= {**cosmo_params, **nuisance_params}

print(cosmo_params)


# info1 = {
#     "likelihood": {"soliket.mflike.MFLike": {"sim_id": 0},},
#     "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1}}},
#     "params": {**cosmo_params, **nuisance_params},
# }



{'cosmomc_theta': 0.0104085, 'As': 2.0989031673191437e-09, 'ombh2': 0.02237, 'omch2': 0.12, 'ns': 0.9649, 'Alens': 1.0, 'tau': 0.0544}


SOLikeT will download the data file as appropriate and store it inside the cobaya packages path. This should only happen once the first time the likelihood is called.

In [70]:
# Make an instance of the model in cobaya from the information provided above
model_mflike = get_model(info)
# What is inside the theory part of this model? We should have a primary CAMB theory function 
# that will yield Cls, and a foreground model
# THESE CONTAIN THE SAME THING???
print(model_mflike.loglike(cosmo_params))
print(model_mflike.loglikes(nuisance_params))

[model] *WARNING* Ignored blocks/options: ['defaults']
[camb] `camb` module loaded successfully from /opt/anaconda3/envs/cobaya-env/lib/python3.9/site-packages/camb
[soliket.mflike.mflike] Initialized!
(-1407.3398619303014, [])
(array([-1407.33986193]), [])


We want to have the saved $\chi^2$ values to compare against, and so we read them in here 

In [65]:
if Version(camb.__version__) >= Version('1.4'):
    chi2s = {"tt": 545.1257,
             "te": 137.4146,
             "ee": 167.9850,
             "tt-te-et-ee": 790.5121}
else:
    chi2s = {"tt": 544.9745,
             "te-et": 152.6807,
             "ee": 168.0953,
             "tt-te-et-ee": 790.4124}

Now we call the mflike likelihood for our cosmological model ad compare it to the $\chi^2$ values above

In [95]:
mflike_compare = model_mflike.likelihood['soliket.mflike.MFLike']
# Let's compute the chi2 value as the loglike of the model but make sure to i
chi2 = - 2* (model_mflike.loglike(nuisance_params)[0] - mflike_compare.logp_const)
print(chi2)
print(f'Model mflike loglike= {chi2:.2f}, compared to saved vals: {2*chi2s["tt-te-et-ee"]:.2f}')
#print(mflike_compare.loglike)

# THESE DO NOT MATCH -- NEED TO CHECK THIS

2434.3466110409036
Model mflike loglike= 2434.35, compared to saved vals: 1581.02


Next up, we want to run the lensing likelihood inside SOLikeT

In [96]:
info_lensing = {
    "likelihood": {"soliket.LensingLikelihood": {"theory_lmax": 5000},},
    "theory": {"camb": {"extra_args": {"lens_potential_accuracy": 1, "kmax": 0.9}}},
    "params": {**cosmo_params},
}


model_lens = get_model(info_lensing)



[camb] `camb` module loaded successfully from /opt/anaconda3/envs/cobaya-env/lib/python3.9/site-packages/camb
[soliket.lensinglikelihood] Initialising.
[camb] `camb` module loaded successfully from /opt/anaconda3/envs/cobaya-env/lib/python3.9/site-packages/camb


In [80]:
lens_compare = model_lens.likelihood['soliket.LensingLikelihood']
print(f'The computed lensing likelihood for params is {model_lens.loglike(cosmo_params)[0]:.2f}, and the saved best fit point is {lens_compare.current_logp:.2f}')

The computed lensing likelihood for params is 324.65, and the saved best fit point is 324.65


Now let us compute the combined multi-probe likelihood (with a Gaussian covariance matrix for now!)

In [97]:
info_multi = {
    "likelihood": {
        "soliket.gaussian.MultiGaussianLikelihood": {
             "components": ["soliket.mflike.MFLike", "soliket.LensingLikelihood"],
             "stop_at_error": True,
             "options": {
                "input_file": 'data_sacc_00044.fits',
                "cov_Bbl_file": "data_sacc_w_covar_and_Bbl.fits",
                "defaults": { 
                    "polarizations": {'TT', 'TE', 'ET', 'EE'},
                    "scales": {
                        "TT": '[2, 6002]',
                        "TE": '[2, 6002]',
                        "ET": '[2, 6002]',
                        "EE": '[2, 6002]'}, 
                        "symmetrize": 'False'
                            },
                "data": {
                        "experiments":{
                                "LAT": {
                                        "frequencies": [93, 145, 225]
                                        }
                                        },
                        "spectra": {
                                {"experiments": ['LAT', 'LAT'], "frequencies": [93, 93], "polarizations": ['TT','TE','EE']},
                                {"experiments": ['LAT', 'LAT'],"frequencies": [93, 145]},
                                {"experiments": ['LAT', 'LAT'], "frequencies": [93, 225]}, 
                                {"experiments": ['LAT', 'LAT'], "frequencies": [145, 145], "polarizations": ['TT','TE','EE']},
                                {"experiments": ['LAT', 'LAT'],"frequencies": [145, 225]},
                                {"experiments": ['LAT', 'LAT'], "frequencies": [225, 225], "polarizations": ['TT','TE','EE']}
                                }
                        }
                        } 
                        }
                        }, 
    "params": {**cosmo_params, **nuisance_params}
    } 



# Make sure that all needed theory components are included here. 
# We can also add them into a new sub-dict of the main info_multi dictionary
info_multi["theory"]= {
    "camb": {"extra_args": {"lens_potential_accuracy": 1}}
}

TypeError: unhashable type: 'dict'

In [91]:
model_multi = get_model(info_multi)
#multi_compare = model_multi.likelihood['soliket.gaussian.MultiGaussianLikelihood']

[model] *WARNING* Ignored blocks/options: ['options']
[camb] `camb` module loaded successfully from /opt/anaconda3/envs/cobaya-env/lib/python3.9/site-packages/camb


TypeError: 'NoneType' object is not iterable